# V4_2系列の総合分析(tontal_analysis)スクリプト
・今の区間から前４つの区間のfinalありモデリングスコアシートをもとに性能の特性を調べる総合分析を行い，csvを出力する(大まかなパラメータ決めのため)

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt

import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
#自作のモジュールのインポート
sys.path.append("../..")
import module.master as master
import module.modeling_scores as modeling


### 関数

In [2]:
def get_season_date(now_date):#日付(datetime型)を渡すと，その日付で購買予測を行う際に使用するデータの区間を返す関数
    use_data_year=now_date.year
    if (now_date.month>=1)and(now_date.month<4):
        use_data_month=1
    elif (now_date.month>=4)and(now_date.month<7):
        use_data_month=4
    elif (now_date.month>=7)and(now_date.month<10):
        use_data_month=7
    elif (now_date.month>=10):
        use_data_month=10
    else:
        print('what???????')
    use_data_date=datetime.datetime(year=use_data_year, month=use_data_month,day=1)
    return use_data_date


def get_4_section_dt(now_date):#今いる区間から直近4区間の開始日をリストで返してくれる関数
    now_sec_date=get_season_date(now_date)
    diff_sec_stdates=[0]*4
    for i in range(len(diff_sec_stdates)):
        diff_sec_stdates[3-i]=now_sec_date- relativedelta(months=3*(i+1))#古い順に日付を入れていく
    return diff_sec_stdates


def gainth_analysis(score_df,test_type='final'):
    
    gainth_arr=[100,110,120,140,160]
    index_arr=[0,1,2,3,4]
    cols=["gain_th","num_com","num_model(row)"]
    gainth_df=pd.DataFrame(columns=cols)
    def make_series(target_df):
        num_com=len(target_df['target_com'].value_counts().index)
        num_model=len(target_df)
        return num_com,num_model
    for gainth,i in zip(gainth_arr,index_arr):
        append_s=pd.Series(index=cols,dtype='object')
        if (i==0):
            target_df=score_df[score_df["gain_{}".format(test_type)]<gainth]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="100未満"
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        elif (i==4):
            target_df=score_df[score_df["gain_{}".format(test_type)]>=gainth]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="150以上"
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        else:
            target_df=score_df[(score_df["gain_{}".format(test_type)]<gainth_arr[i])&(score_df["gain_{}".format(test_type)]>=gainth_arr[i-1])]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="{}以上_{}未満".format(gainth_arr[i-1],gainth_arr[i])
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        gainth_df=gainth_df.append(append_s,ignore_index=True)
    return gainth_df

def total_analysis_recent(now_date,place_names,version,plus_m_th,gain_ths,gain_std_ths,hit_per_ths,hit_std_ths,num_hit_ths):#直近４区間のモデリングスコアをもとに総合分析を行う関数
    mode_type='recent'
    mode='model_score_'
    
    diff_sec_dates=get_4_section_dt(now_date)#直近４区間の開始日をリストで取得
    now_date_txt=now_date.strftime('%Y%m%d')
    d1_sec_date_txt=diff_sec_dates[0].strftime('%Y%m%d')
    d2_sec_date_txt=diff_sec_dates[1].strftime('%Y%m%d')
    d3_sec_date_txt=diff_sec_dates[2].strftime('%Y%m%d')
    d4_sec_date_txt=diff_sec_dates[3].strftime('%Y%m%d')
    print('version:{}_______mode:{}==================================================='.format(version,mode_type))
    for place_name in tqdm(place_names):
        analysis_v_df=pd.DataFrame(columns=['minus_model_per_mean','num_plus_com_mean','num_plus_model_mean','gain_th','gain_std_th','hit_per_th','hit_std_th'])
        #こっから下はバージョン，モードごと
        for num_hit_th in num_hit_ths:
            for gain_th in gain_ths:
                for gain_std_th in gain_std_ths:
                    for hit_per_th in hit_per_ths:
                        for hit_std_th in hit_std_ths:
                            append_s=pd.Series(dtype='object')
                            #for i in range(4):
                            for sec_date in diff_sec_dates:#区間の日付でfor文を回す
                                #now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
                                sec_date_txt=sec_date.strftime('%Y%m%d')
                                dir_path='../../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{sec_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,sec_ym=sec_date_txt,version=version)
                                model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
                                model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
                                #決めのパラメータでいったん切り落とし
                                model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
                                model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる,これも可変していく方針に変更した
                                #可変していくパラメータ
                                target_df=model_score_df[model_score_df['gain_test']>=gain_th].copy()
                                target_df=target_df[target_df['gain_std_test']<=gain_std_th].copy()
                                target_df=target_df[target_df['buy_hit_per_test']>=hit_per_th].copy()
                                target_df=target_df[target_df['buy_hit_per_std_test']<=hit_std_th].copy()
                                analysis_df=gainth_analysis(target_df,test_type='final')
                                minus_df=analysis_df.loc[:'100以上_110未満'].copy()
                                plus_df=analysis_df.loc['100以上_110未満':].copy()
                                try:
                                    minus_model_per=(minus_df['num_model(row)'].sum()/analysis_df['num_model(row)'].sum())*100
                                except ZeroDivisionError:
                                    minus_model_per=100
                                append_s['st_{}_minus_per(model)'.format(sec_date_txt)]=minus_model_per
                                append_s['st_{}_num_plus_com'.format(sec_date_txt)]=plus_df['num_com'].sum()
                                append_s['st_{}_num_plus_model(row)'.format(sec_date_txt)]=plus_df['num_model(row)'].sum()
                            append_s['num_hit_th']=num_hit_th
                            append_s['gain_th']=gain_th
                            append_s['gain_std_th']=gain_std_th
                            append_s['hit_per_th']=hit_per_th
                            append_s['hit_std_th']=hit_std_th
                            analysis_v_df=analysis_v_df.append(append_s,ignore_index=True)
        analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_{}_minus_per(model)'.format(d1_sec_date_txt)]+analysis_v_df['st_{}_minus_per(model)'.format(d2_sec_date_txt)]+analysis_v_df['st_{}_minus_per(model)'.format(d3_sec_date_txt)]+analysis_v_df['st_{}_minus_per(model)'.format(d4_sec_date_txt)])/4)        
        analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_{}_num_plus_com'.format(d1_sec_date_txt)]+analysis_v_df['st_{}_num_plus_com'.format(d2_sec_date_txt)]+analysis_v_df['st_{}_num_plus_com'.format(d3_sec_date_txt)]+analysis_v_df['st_{}_num_plus_com'.format(d4_sec_date_txt)])/4)        
        analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_{}_num_plus_model(row)'.format(d1_sec_date_txt)]+analysis_v_df['st_{}_num_plus_model(row)'.format(d2_sec_date_txt)]+analysis_v_df['st_{}_num_plus_model(row)'.format(d3_sec_date_txt)]+analysis_v_df['st_{}_num_plus_model(row)'.format(d4_sec_date_txt)])/4)        
        #csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{version}_{mode_type}_TRhit_pM{plus_month}_st{now_ym}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version,plus_month=plus_m_th,num_hit=num_hit_th,now_ym=now_date_txt)
        csv_path='../../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{version}_{mode_type}_TRhit_pM{plus_month}_st{now_ym}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version,plus_month=plus_m_th,num_hit=num_hit_th,now_ym=now_date_txt)
        analysis_v_df.to_csv(csv_path)
    return None

def total_analysis_period(now_date,place_names,version,plus_m_th,gain_ths,gain_std_ths,hit_per_ths,hit_std_ths,num_hit_ths):#直近４区間のモデリングスコアをもとに総合分析を行う関数
    mode_type='period'
    mode='model_score_period_'
    
    diff_sec_dates=get_4_section_dt(now_date)#直近４区間の開始日をリストで取得
    now_date_txt=now_date.strftime('%Y%m%d')
    d1_sec_date_txt=diff_sec_dates[0].strftime('%Y%m%d')
    d2_sec_date_txt=diff_sec_dates[1].strftime('%Y%m%d')
    d3_sec_date_txt=diff_sec_dates[2].strftime('%Y%m%d')
    d4_sec_date_txt=diff_sec_dates[3].strftime('%Y%m%d')
    print('version:{}_______mode:{}==================================================='.format(version,mode_type))
    for place_name in tqdm(place_names):
        analysis_v_df=pd.DataFrame(columns=['minus_model_per_mean','num_plus_com_mean','num_plus_model_mean','gain_th','gain_std_th','hit_per_th','hit_std_th'])
        #こっから下はバージョン，モードごと
        for num_hit_th in num_hit_ths:
            for gain_th in gain_ths:
                for gain_std_th in gain_std_ths:
                    for hit_per_th in hit_per_ths:
                        for hit_std_th in hit_std_ths:
                            append_s=pd.Series(dtype='object')
                            #for i in range(4):
                            for sec_date in diff_sec_dates:#区間の日付でfor文を回す
                                #now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1).strftime('%Y%m%d')
                                sec_date_txt=sec_date.strftime('%Y%m%d')
                                dir_path='../../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{place_name}_{mode}st{sec_ym}_finalM3_{version}.csv'.format(place_name=place_name,mode=mode,sec_ym=sec_date_txt,version=version)
                                model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
                                model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
                                #決めのパラメータでいったん切り落とし
                                model_score_df=model_score_df[model_score_df['plus_month_num_test']>=plus_m_th].copy()#収益がプラスだった月の数で切り捨てる
                                model_score_df=model_score_df[model_score_df['num_hit_test']>=num_hit_th].copy()#的中した予測の数で切り捨てる,これも可変していく方針に変更した
                                #可変していくパラメータ
                                target_df=model_score_df[model_score_df['gain_test']>=gain_th].copy()
                                target_df=target_df[target_df['gain_std_test']<=gain_std_th].copy()
                                target_df=target_df[target_df['buy_hit_per_test']>=hit_per_th].copy()
                                target_df=target_df[target_df['buy_hit_per_std_test']<=hit_std_th].copy()
                                analysis_df=gainth_analysis(target_df,test_type='final')
                                minus_df=analysis_df.loc[:'100以上_110未満'].copy()
                                plus_df=analysis_df.loc['100以上_110未満':].copy()
                                try:
                                    minus_model_per=(minus_df['num_model(row)'].sum()/analysis_df['num_model(row)'].sum())*100
                                except ZeroDivisionError:
                                    minus_model_per=100
                                append_s['st_{}_minus_per(model)'.format(sec_date_txt)]=minus_model_per
                                append_s['st_{}_num_plus_com'.format(sec_date_txt)]=plus_df['num_com'].sum()
                                append_s['st_{}_num_plus_model(row)'.format(sec_date_txt)]=plus_df['num_model(row)'].sum()
                            append_s['num_hit_th']=num_hit_th
                            append_s['gain_th']=gain_th
                            append_s['gain_std_th']=gain_std_th
                            append_s['hit_per_th']=hit_per_th
                            append_s['hit_std_th']=hit_std_th
                            analysis_v_df=analysis_v_df.append(append_s,ignore_index=True)
        analysis_v_df['minus_model_per_mean']=((analysis_v_df['st_{}_minus_per(model)'.format(d1_sec_date_txt)]+analysis_v_df['st_{}_minus_per(model)'.format(d2_sec_date_txt)]+analysis_v_df['st_{}_minus_per(model)'.format(d3_sec_date_txt)]+analysis_v_df['st_{}_minus_per(model)'.format(d4_sec_date_txt)])/4)        
        analysis_v_df['num_plus_com_mean']=((analysis_v_df['st_{}_num_plus_com'.format(d1_sec_date_txt)]+analysis_v_df['st_{}_num_plus_com'.format(d2_sec_date_txt)]+analysis_v_df['st_{}_num_plus_com'.format(d3_sec_date_txt)]+analysis_v_df['st_{}_num_plus_com'.format(d4_sec_date_txt)])/4)        
        analysis_v_df['num_plus_model_mean']=((analysis_v_df['st_{}_num_plus_model(row)'.format(d1_sec_date_txt)]+analysis_v_df['st_{}_num_plus_model(row)'.format(d2_sec_date_txt)]+analysis_v_df['st_{}_num_plus_model(row)'.format(d3_sec_date_txt)]+analysis_v_df['st_{}_num_plus_model(row)'.format(d4_sec_date_txt)])/4)        
        #csv_path='../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{version}_{mode_type}_TRhit_pM{plus_month}_st{now_ym}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version,plus_month=plus_m_th,num_hit=num_hit_th,now_ym=now_date_txt)
        csv_path='../../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/{version}_{mode_type}_TRhit_pM{plus_month}_st{now_ym}_total_analysis.csv'.format(place_name=place_name,mode_type=mode_type,version=version,plus_month=plus_m_th,num_hit=num_hit_th,now_ym=now_date_txt)
        analysis_v_df.to_csv(csv_path)
    return None

## total_analysis

#### pm=3

In [3]:
# #today=datetime.date.today()#今日の日付の取得
# today=datetime.datetime(year=2021, month=1,day=1)

# diff_sec_dates=get_4_section_dt(today)#直近４区間の開始日をリストで取得
# #diff_sec_dates=get_4_section_dt(st_ym)#直近４区間の開始日をリストで取得

# place_master=master.get_place_master()
# place_names=[place_name for place_name in place_master.values()]#会場名のみを収納した配列
# version='V4_2'
# #決めパラメータ，（今のところ月のみ）
# plus_m_th=3
# #きめじゃないパラメータの範囲設定(ループの数に注意！！)
# gain_ths=np.arange(80, 180,4)
# gain_std_ths=np.arange(10, 250,15)
# hit_per_ths=np.arange(0,20,2)
# hit_std_ths=np.arange(0, 2.0, 0.4)
# num_hit_ths=[3,5,8,10,13,15]



# time_sta = time.time()
# #========
# #総合分析シートの出力
# total_analysis_recent(now_date,place_names,version,plus_m_th,gain_ths,gain_std_ths,hit_per_ths,hit_std_ths,num_hit_ths)#直近４区間のモデリングスコアをもとに総合分析を行う関数
# #========

# time_end = time.time()
# # 経過時間（秒）
# tim = time_end- time_sta

# print('DONE')
# print('elapsed time::{}=================================================================================================='.format(tim))

#### pm=1

In [ ]:
#today=datetime.date.today()#今日の日付の取得
today=datetime.datetime(year=2021, month=1,day=1)

diff_sec_dates=get_4_section_dt(today)#直近４区間の開始日をリストで取得
#diff_sec_dates=get_4_section_dt(st_ym)#直近４区間の開始日をリストで取得

place_master=master.get_place_master()
place_names=[place_name for place_name in place_master.values()]#会場名のみを収納した配列
#place_names=place_names[:12]
place_names=place_names[9:12]

version='V4_2'
#決めパラメータ，（今のところ月のみ）
plus_m_th=1
#きめじゃないパラメータの範囲設定(ループの数に注意！！)
gain_ths=np.arange(80, 180,5)
gain_std_ths=np.arange(10, 250,15)
hit_per_ths=np.arange(0,20,2)
hit_std_ths=np.arange(0, 2.0, 0.5)
num_hit_ths=[3,5,8,10,13]

print(place_names)


time_sta = time.time()
#========
#総合分析シートの出力
total_analysis_period(today,place_names,version,plus_m_th,gain_ths,gain_std_ths,hit_per_ths,hit_std_ths,num_hit_ths)#直近４区間のモデリングスコアをもとに総合分析を行う関数
#========

time_end = time.time()
# 経過時間（秒）
tim = time_end- time_sta

print('DONE')
print('elapsed time::{}=================================================================================================='.format(tim))

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

['mikuni', 'biwako', 'suminoe']
version:V4_2_______mode:period===================================================


In [ ]:
# today=datetime.date.today()#今日の日付の取得
# diff_sec_dates=get_4_section_dt(today)#直近４区間の開始日をリストで取得
# diff_sec_dates